In [1]:
import pandas as pd
import numpy as np
import itertools

In [2]:
# reading file
data = pd.read_csv("fifa21_train.csv")
pd.set_option('display.max_columns', None)

# Turn everying to snake_case
data.columns = list(map(lambda x: x.lower(), data.columns))
data.columns = data.columns.str.replace(' ','_')
data.head(5)

,id,name,age,nationality,club,bp,position,team_&_contract,height,weight,foot,growth,joined,loan_date_end,value,wage,release_clause,contract,attacking,crossing,finishing,heading_accuracy,short_passing,volleys,skill,dribbling,curve,fk_accuracy,long_passing,ball_control,movement,acceleration,sprint_speed,agility,reactions,balance,power,shot_power,jumping,stamina,strength,long_shots,mentality,aggression,interceptions,positioning,vision,penalties,composure,defending,marking,standing_tackle,sliding_tackle,goalkeeping,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes,total_stats,base_stats,w/f,sm,a/w,d/w,ir,pac,sho,pas,dri,def,phy,hits,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,ova
0,184383,A. Pasche,26,Switzerland,FC Lausanne-Sport,CM,CM CDM,FC Lausanne-Sport 2015 ~ 2020,"5'9""",161lbs,Right,1,"Jul 1, 2015",NaN,€525K,€4K,€801K,2015 ~ 2020,258,54,47,43,70,44.0,286,61,44.0,55,63,63,346,64,73,61.0,66,82.0,306,62,73.0,71,55,45,290,54,52.0,62.0,68.0,54,54.0,148,49,56,43.0,48,7,12,14,9,6,1682,357,4 ★,2★,High,Medium,1 ★,69,51,63,63,51,60,3,58+1,58+1,58+1,61+0,62+0,62+0,62+0,61+0,63+1,63+1,63+1,63+1,63+1,63+1,63+1,63+1,59+1,59+1,59+1,59+1,59+1,58+1,54+1,54+1,54+1,58+1,15+1,64
1,188044,Alan Carvalho,30,China PR,Beijing Sinobo Guoan FC,ST,ST LW LM,"Beijing Sinobo Guoan FC Dec 31, 2020 On Loan","6'0""",159lbs,Right,0,"Jan 16, 2015","Dec 31, 2020",€8.5M,€23K,€0,"Dec 31, 2020 On Loan",365,66,79,76,68,76.0,375,83,78.0,72,63,79,404,83,83,88.0,75,75.0,372,74,81.0,75,74,68,313,54,33.0,78.0,72.0,76,70.0,77,35,20,22.0,55,11,7,14,7,16,1961,412,3 ★,4★,High,Low,2 ★,83,75,68,82,33,71,44,77+0,77+0,77+0,77+0,77+0,77+0,77+0,77+0,76+1,76+1,76+1,76+1,68+2,68+2,68+2,76+1,57+2,53+2,53+2,53+2,57+2,53+2,48+2,48+2,48+2,53+2,18+2,77
2,184431,S. Giovinco,33,Italy,Al Hilal,CAM,CAM CF,Al Hilal 2019 ~ 2022,"5'4""",134lbs,Right,0,"Jan 31, 2019",NaN,€9M,€49K,€15.3M,2019 ~ 2022,336,73,76,34,78,75.0,424,85,89.0,91,74,85,424,84,76,93.0,78,93.0,308,79,34.0,75,42,78,332,75,26.0,80.0,78.0,73,82.0,80,23,29,28.0,21,6,3,6,3,3,1925,404,4 ★,4★,High,Medium,2 ★,80,77,78,86,27,56,73,73+2,73+2,73+2,80+0,79+0,79+0,79+0,80+0,80+0,80+0,80+0,79+1,74+2,74+2,74+2,79+1,59+2,56+2,56+2,56+2,59+2,53+2,41+2,41+2,41+2,53+2,12+2,80
3,233796,J. Evans,22,Wales,Swansea City,CDM,CDM CM,Swansea City 2016 ~ 2021,"5'10""",152lbs,Right,13,"Jul 1, 2016",NaN,€275K,€4K,€694K,2016 ~ 2021,242,44,42,58,62,36.0,259,54,41.0,46,57,61,282,54,59,59.0,55,55.0,277,57,60.0,64,58,38,257,61,57.0,31.0,54.0,54,48.0,168,55,58,55.0,42,8,9,6,7,12,1527,329,2 ★,2★,Medium,Medium,1 ★,57,44,54,57,57,60,7,50+2,50+2,50+2,51+0,51+0,51+0,51+0,51+0,53+2,53+2,53+2,53+2,56+2,56+2,56+2,53+2,56+2,58+2,58+2,58+2,56+2,57+2,58+2,58+2,58+2,57+2,14+2,59
4,234799,Y. Demoncy,23,France,US Orléans Loiret Football,CDM,CDM CM,US Orléans Loiret Football 2018 ~ 2021,"5'11""",150lbs,Right,8,"Jul 1, 2018",NaN,€725K,€2K,€1.4M,2018 ~ 2021,249,49,37,61,68,34.0,280,64,44.0,45,61,66,324,66,66,61.0,62,69.0,280,61,34.0,81,61,43,294,66,60.0,55.0,64.0,49,58.0,185,58,61,66.0,52,8,9,15,5,15,1664,360,2 ★,3★,Low,Medium,1 ★,66,44,60,64,60,66,4,56+2,56+2,56+2,59+0,59+0,59+0,59+0,59+0,61+2,61+2,61+2,62+2,63+2,63+2,63+2,62+2,64+2,64+2,64+2,64+2,64+2,63+2,61+2,61+2,61+2,63+2,15+2,65


In [3]:
def removing_columns(data,indexes1,indexes2):
    # Iteration that removes from column index1 to column index2 from dataframe
    for i in range(len(indexes1)):
        cols = list(data.columns)
        min_index = cols.index(indexes1[i])
        max_index = cols.index(indexes2[i])
        data = data.drop(cols[min_index:max_index+1],axis=1)    
    return data

# Deleting sub_scores
removing_index1 = ["ls","gk_diving","marking","aggression","shot_power","acceleration","dribbling","crossing","pac"]
removing_index2 = ["gk","gk_reflexes","sliding_tackle","penalties","long_shots","balance","ball_control","volleys","phy"]
data = removing_columns(data,removing_index1,removing_index2)

In [4]:
# dropping useless columns
data = data.drop(['id','name','team_&_contract','contract','loan_date_end','joined','position','club'],axis=1)
data = data.drop(['nationality'],axis=1)
# Dropping na
data = data.dropna()


In [5]:
def money_to_num(data,columns):
    # Removes € and transform K,M to numerical.
    for column in columns:
        data[column] = data[column].str.replace("€", "").str.replace("K", "*1000").str.replace("M", "*1000000").apply(eval)
    return data

def remove_stars(data,stars):
    # Removes ★ from "stars" column
    for column in stars:
        data[column] = data[column].str.replace("★", "")
        data[column] = pd.to_numeric(data[column], errors='coerce') 
    return data

def labeling(column,old_names,labeling):
    labels = list(range(labeling[0],len(old_names)+1,labeling[1]))
    for i in range(len(old_names)):
        column = column.str.replace(old_names[i],str(labels[i]))
    column = pd.to_numeric(column)
    return column

# Weight to numeric.
data['weight'] = pd.to_numeric(data['weight'].str.replace("lbs", ""), errors='coerce') 

# Height to numeric.
data['height'] = data['height'].str.replace("'", "+").str.replace('"', "/12").apply(eval)

# Wage,Value,Release clause and Hits to numeric.
data = money_to_num(data,['wage','value','hits','release_clause'])
#hits it's not money but you can use the same function

# Removing stars from columns
data = remove_stars(data,['w/f','sm','ir'])

# a/w and d/w labeling to 1,2,3
data['a/w'] = labeling(data['a/w'],['Low','Medium','High'],[1,1])
data['d/w'] = labeling(data['d/w'],['Low','Medium','High'],[1,1])

# Filtering wage <= 0
data = data[data['wage']>0]

In [6]:
data_cat = data.select_dtypes(object)
data_num = data.select_dtypes(np.number)

In [7]:
# We take out X numerical columns that have high correlation (<0.9) between them
for X_column1 in data_num.columns[:-1]:
    for X_column2 in data_num.columns[:-1]:
        if X_column1 != X_column2 and X_column2 in data.columns and X_column1 in data.columns and abs(data[X_column1].corr(data[X_column2]))>0.9:
            #data=data.drop([X_column2],axis=1)
            print("Variables "+data[X_column1].name + " and " +data[X_column2].name + " have a corr of " + str(data[X_column1].corr(data[X_column2])))

Variables value and release_clause have a corr of 0.9802833055930664
Variables release_clause and value have a corr of 0.9802833055930664
Variables attacking and skill have a corr of 0.9287777160246722
Variables attacking and total_stats have a corr of 0.9045009248200865
Variables skill and attacking have a corr of 0.9287777160246722
Variables skill and total_stats have a corr of 0.9296130903783199
Variables mentality and total_stats have a corr of 0.9490913375367084
Variables total_stats and attacking have a corr of 0.9045009248200866
Variables total_stats and skill have a corr of 0.9296130903783199
Variables total_stats and mentality have a corr of 0.9490913375367084


In [8]:
# We remove release clause and total stats because it has high correlation with many variables.
data = data.drop(["total_stats","release_clause","attacking"],axis=1)

# We take out X numerical columns that have high correlation (<0.9) between them
for X_column1 in data_num.columns[:-1]:
    for X_column2 in data_num.columns[:-1]:
        if X_column1 != X_column2 and X_column2 in data.columns and X_column1 in data.columns and abs(data[X_column1].corr(data[X_column2]))>0.9:
            #data=data.drop([X_column2],axis=1)
            print("Variables "+data[X_column1].name + " and " +data[X_column2].name + " have a corr of " + str(data[X_column1].corr(data[X_column2])))

In [9]:
# X-y split
X = data.drop('ova',axis=1)
y = data['ova']

# Normalize
from sklearn.preprocessing import MinMaxScaler 
X_num = X.select_dtypes(np.number)
transformer = MinMaxScaler().fit(X_num)
X_normalized = transformer.transform(X_num)
X_normalized = pd.DataFrame(X_normalized, columns=X_num.columns)
X_normalized.describe().T

,count,mean,std,min,25%,50%,75%,max
age,11190.0,0.336342,0.179783,0.0,0.185185,0.296296,0.444444,1.0
height,11190.0,0.517560,0.135828,0.0,0.400000,0.500000,0.600000,1.0
weight,11190.0,0.384909,0.125280,0.0,0.293651,0.380952,0.468254,1.0
growth,11190.0,0.218196,0.224537,0.0,0.000000,0.153846,0.384615,1.0
value,11190.0,0.029310,0.059541,0.0,0.004444,0.009444,0.028889,1.0
wage,11190.0,0.017920,0.038118,0.0,0.002681,0.006256,0.018767,1.0
skill,11190.0,0.522751,0.179268,0.0,0.442623,0.550351,0.644028,1.0
movement,11190.0,0.598839,0.156494,0.0,0.518519,0.623932,0.706553,1.0
power,11190.0,0.544207,0.159303,0.0,0.443730,0.562701,0.662379,1.0
mentality,11190.0,0.574904,0.174501,0.0,0.498607,0.596100,0.690808,1.0


In [10]:
#One Hot/Label (categorical).
from sklearn.preprocessing import OneHotEncoder
X_cat = X.select_dtypes(object)
encoder = OneHotEncoder(drop='first').fit(X_cat)
encoded = encoder.transform(X_cat).toarray()
onehot_encoded = pd.DataFrame(encoded,columns=encoder.get_feature_names_out(X_cat.columns))

In [11]:
onehot_encoded

,bp_CB,bp_CDM,bp_CF,bp_CM,bp_GK,bp_LB,bp_LM,bp_LW,bp_LWB,bp_RB,bp_RM,bp_RW,bp_RWB,bp_ST,foot_Right
0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11185,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
11186,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11187,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
11188,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0


In [12]:
X_ready = pd.concat([X_normalized,onehot_encoded],axis=1)
X_ready

,age,height,weight,growth,value,wage,skill,movement,power,mentality,composure,defending,goalkeeping,base_stats,w/f,sm,a/w,d/w,ir,hits,bp_CB,bp_CDM,bp_CF,bp_CM,bp_GK,bp_LB,bp_LM,bp_LW,bp_LWB,bp_RB,bp_RM,bp_RW,bp_RWB,bp_ST,foot_Right
0,0.370370,0.40,0.349206,0.038462,0.005833,0.006256,0.569087,0.663818,0.556270,0.654596,0.500000,0.518219,0.084309,0.492366,0.75,0.25,1.0,0.5,0.00,0.000000,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.518519,0.55,0.333333,0.000000,0.094444,0.040214,0.777518,0.829060,0.768489,0.718663,0.690476,0.230769,0.100703,0.702290,0.50,0.75,1.0,0.0,0.25,0.004458,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
2,0.629630,0.15,0.134921,0.000000,0.100000,0.086685,0.892272,0.886040,0.562701,0.771588,0.833333,0.242915,0.021077,0.671756,0.75,0.75,1.0,0.5,0.25,0.007611,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.222222,0.45,0.277778,0.500000,0.003056,0.006256,0.505855,0.481481,0.463023,0.562674,0.428571,0.599190,0.070258,0.385496,0.25,0.25,0.5,0.5,0.00,0.000435,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.259259,0.50,0.261905,0.307692,0.008056,0.002681,0.555035,0.601140,0.472669,0.665738,0.547619,0.668016,0.093677,0.503817,0.25,0.50,0.0,0.5,0.00,0.000109,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11185,0.481481,0.30,0.293651,0.000000,0.006111,0.000000,0.562061,0.726496,0.546624,0.554318,0.571429,0.242915,0.112412,0.416031,0.50,0.50,1.0,0.0,0.00,0.000109,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
11186,0.333333,0.60,0.404762,0.192308,0.006944,0.004468,0.480094,0.626781,0.450161,0.587744,0.404762,0.663968,0.114754,0.454198,0.50,0.25,0.5,0.5,0.00,0.000000,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11187,0.407407,0.60,0.452381,0.153846,0.017778,0.006256,0.124122,0.353276,0.270096,0.128134,0.523810,0.085020,0.784543,0.606870,0.25,0.00,0.5,0.5,0.00,0.000000,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
11188,0.222222,0.25,0.206349,0.307692,0.012222,0.002681,0.580796,0.783476,0.466238,0.518106,0.547619,0.170040,0.105386,0.416031,0.50,0.50,1.0,0.5,0.00,0.000326,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0


In [13]:
from sklearn.metrics import mean_squared_error, r2_score,mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn import linear_model

# Splitting model
X_train, X_test, y_train, y_test = train_test_split(X_ready, y, test_size=0.2, random_state=42)

In [14]:
# Training model with train data
lm = linear_model.LinearRegression()
lm.fit(X_train,y_train)

# Comparing values of Train and Test
# Checking the value of R^2
predictions = lm.predict(X_train)
print('The train R^2 is %1.3f.' %r2_score(y_train, predictions))
test_predictions = lm.predict(X_test)
print('The test R^2 is %1.3f.' %r2_score(y_test,test_predictions))

# Checking the value of MSE
print('\nThe train MSE is %i.1' %mean_squared_error(y_train,predictions))
print('The test MSE is %i.1' %mean_squared_error(y_test,test_predictions))

# Checking the value of RMSE
print('\nThe train RMSE is %1.1f.' %np.sqrt(mean_squared_error(y_train,predictions)))
print('The test RMSE is %1.1f.' %np.sqrt(mean_squared_error(y_test,test_predictions)))

# Checking the value of MAE
print('\nThe train MAE is %1.1f.'%mean_absolute_error(y_train,predictions))
print('The test MAE is %1.1f.'%mean_absolute_error(y_test,test_predictions))

The train R^2 is 0.874.
The test R^2 is 0.877.

The train MSE is 5.1
The test MSE is 5.1

The train RMSE is 2.4.
The test RMSE is 2.4.

The train MAE is 1.9.
The test MAE is 1.9.


In [15]:
X_test

,age,height,weight,growth,value,wage,skill,movement,power,mentality,composure,defending,goalkeeping,base_stats,w/f,sm,a/w,d/w,ir,hits,bp_CB,bp_CDM,bp_CF,bp_CM,bp_GK,bp_LB,bp_LM,bp_LW,bp_LWB,bp_RB,bp_RM,bp_RW,bp_RWB,bp_ST,foot_Right
9979,0.259259,0.40,0.293651,0.269231,0.004722,0.000000,0.543326,0.686610,0.578778,0.534819,0.583333,0.469636,0.079625,0.492366,0.75,0.50,1.0,0.5,0.00,0.000109,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4333,0.629630,0.55,0.420635,0.000000,0.012222,0.040214,0.573770,0.643875,0.688103,0.718663,0.619048,0.821862,0.105386,0.660305,0.50,0.25,0.5,1.0,0.25,0.000326,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
7910,0.074074,0.55,0.420635,0.653846,0.008333,0.002681,0.648712,0.598291,0.700965,0.593315,0.690476,0.433198,0.091335,0.549618,0.50,0.50,0.5,0.0,0.00,0.003371,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4395,0.111111,0.40,0.230159,0.576923,0.042222,0.013405,0.690867,0.829060,0.472669,0.473538,0.452381,0.376518,0.074941,0.557252,0.50,0.50,1.0,0.5,0.00,0.076764,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
3016,0.666667,0.55,0.380952,0.000000,0.006111,0.009830,0.754098,0.595442,0.610932,0.735376,0.666667,0.352227,0.077283,0.541985,0.75,0.50,0.5,0.5,0.25,0.000544,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10971,0.666667,0.45,0.404762,0.000000,0.016667,0.022341,0.744731,0.518519,0.633441,0.785515,0.821429,0.740891,0.086651,0.622137,0.50,0.50,0.5,1.0,0.50,0.001740,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
447,0.222222,0.60,0.492063,0.307692,0.009444,0.006256,0.367681,0.438746,0.495177,0.509749,0.476190,0.716599,0.110070,0.324427,0.50,0.25,0.5,0.5,0.00,0.000435,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3819,0.185185,0.55,0.507937,0.461538,0.133333,0.034853,0.587822,0.683761,0.704180,0.640669,0.726190,0.251012,0.084309,0.549618,0.50,0.50,1.0,0.5,0.00,0.000870,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
56,0.666667,0.65,0.468254,0.000000,0.002222,0.004468,0.498829,0.350427,0.485531,0.598886,0.559524,0.712551,0.077283,0.396947,0.50,0.25,0.5,1.0,0.00,0.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
def everything_again(dataframe):
    # Turn everying to snake_case
    data = dataframe
    data.columns = list(map(lambda x: x.lower(), data.columns))
    data.columns = data.columns.str.replace(' ','_')

    # Deleting sub_scores
    removing_index1 = ["ls","gk_diving","marking","aggression","shot_power","acceleration","dribbling","crossing","pac"]
    removing_index2 = ["gk","gk_reflexes","sliding_tackle","penalties","long_shots","balance","ball_control","volleys","phy"]
    data = removing_columns(data,removing_index1,removing_index2)

    # dropping useless columns
    data = data.drop(['id','name','team_&_contract','contract','loan_date_end','joined','position','club'],axis=1)
    data = data.drop(['nationality'],axis=1)
    # Dropping na
    data = data.dropna()

    # Weight to numeric.
    data['weight'] = pd.to_numeric(data['weight'].str.replace("lbs", ""), errors='coerce') 

    # Height to numeric.
    data['height'] = data['height'].str.replace("'", "+").str.replace('"', "/12").apply(eval)

    # Wage,Value,Release clause and Hits to numeric.
    data = money_to_num(data,['wage','value','hits','release_clause'])
    #hits it's not money but you can use the same function

    # Removing stars from columns
    data = remove_stars(data,['w/f','sm','ir'])

    # a/w and d/w labeling to 1,2,3
    data['a/w'] = labeling(data['a/w'],['Low','Medium','High'],[1,1])
    data['d/w'] = labeling(data['d/w'],['Low','Medium','High'],[1,1])

    # Filtering wage <= 0
    data = data[data['wage']>0]

    data_cat = data.select_dtypes(object)
    data_num = data.select_dtypes(np.number)

    # We remove release clause and total stats because it has high correlation with many variables.
    data = data.drop(["total_stats","release_clause","attacking"],axis=1)

    # X-y split
    X = data.drop('ova',axis=1)
    y = data['ova']

    # Normalize
    from sklearn.preprocessing import MinMaxScaler 
    X_num = X.select_dtypes(np.number)
    transformer = MinMaxScaler().fit(X_num)
    X_normalized = transformer.transform(X_num)
    X_normalized = pd.DataFrame(X_normalized, columns=X_num.columns)

    #One Hot/Label (categorical).
    from sklearn.preprocessing import OneHotEncoder
    X_cat = X.select_dtypes(object)
    encoder = OneHotEncoder(drop='first').fit(X_cat)
    encoded = encoder.transform(X_cat).toarray()
    onehot_encoded = pd.DataFrame(encoded,columns=encoder.get_feature_names_out(X_cat.columns))
    X_ready = pd.concat([X_normalized,onehot_encoded],axis=1)

    return X_ready,y

In [19]:
jam_file = pd.read_csv("fifa21_validate.csv")
jam_X,jam_y = everything_again(jam_file)
jam_predictions = lm.predict(jam_X)

# Comparing values of Train and Test and Jam file
# Checking the value of R^2

print('The train R^2 is %1.3f.' %r2_score(y_train, predictions))
print('The test R^2 is %1.3f.' %r2_score(y_test,test_predictions))
print('The jam test R^2 is %1.3f.' %r2_score(jam_y,jam_predictions))

# Checking the value of RMSE
print('\nThe train RMSE is %1.1f.' %np.sqrt(mean_squared_error(y_train,predictions)))
print('The test RMSE is %1.1f.' %np.sqrt(mean_squared_error(y_test,test_predictions)))
print('The jam test RMSE is %1.1f.' %np.sqrt(mean_squared_error(jam_y,jam_predictions)))


The train R^2 is 0.874.
The test R^2 is 0.877.
The jam test R^2 is 0.786.

The train RMSE is 2.4.
The test RMSE is 2.4.
The jam test RMSE is 3.1.
